# Balanceamento da Classe 0 com GANs ClássicosEste notebook reproduz o pipeline definido em `balance_class0_runs.py`, estruturando-o em célulaspara facilitar a experimentação iterativa. O fluxo completo executa as seguintes etapas:1. Treina um DCGAN para a classe 0 do conjunto escolhido do MedMNIST.2. Gera amostras sintéticas para balancear as classes.3. Treina diversas vezes um classificador ResNet-18 sobre o conjunto balanceado.4. Registra métricas e estatísticas agregadas.Configure os parâmetros na seção **Execução do experimento** para repetir os ensaios desejados.

In [3]:
# Importações futuras
from __future__ import annotations

# Importações padrão
import argparse
import json
import os
import random
import time
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Sequence, Tuple

# Bibliotecas científicas
import numpy as np
import pandas as pd

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import DataLoader, Dataset, TensorDataset

# TorchVision
from torchvision.models import resnet18

# Módulos personalizados
from classical_gans import (
    CGANDiscriminator,
    CGANGenerator,
    DCDiscriminator,
    DCGenerator,
    WGANGPCritic,
    WGANGPGenerator,
    train_cgan,
    train_gan_for_class,
    train_wgangp,
)
from medmnist_data import load_medmnist_data


In [4]:
# ---------------------------------------------------------------------------
# Constantes e dataclasses auxiliares
# ---------------------------------------------------------------------------

# Métricas de classificação
CLASSIFICATION_METRICS = [
    "acc", "prec", "rec", "f1", "auc", "tn", "fp", "fn", "tp"
]

# Métricas de tempo
TIME_METRICS = [
    "gan_training_time_sec",
    "synthetic_generation_time_sec",
    "classifier_training_time_sec",
    "classifier_eval_time_sec",
]

# Campos numéricos adicionais
ADDITIONAL_NUMERIC_FIELDS = [
    "ratio",
    "synthetic_class0_count",
    "balanced_dataset_size",
    "total_real_samples",
    "real_class0_count",
    "real_class1_count",
]

# Campos agregados
AGGREGATION_FIELDS = CLASSIFICATION_METRICS + TIME_METRICS + ADDITIONAL_NUMERIC_FIELDS


# ---------------------------------------------------------------------------
# Dataclasses auxiliares
# ---------------------------------------------------------------------------

from dataclasses import dataclass
from pathlib import Path
from typing import Optional


@dataclass(frozen=True)
class StrategySpec:
    """Define uma estratégia de balanceamento utilizada nos experimentos."""

    key: str
    display_name: str
    uses_gan: bool = False
    conditional_label: Optional[int] = None


BALANCING_STRATEGIES: List[StrategySpec] = [
    StrategySpec("baseline", "ResNet sem balanceamento"),
    StrategySpec("dcgan", "DCGAN (classe 0)", uses_gan=True, conditional_label=0),
    StrategySpec("cgan", "CGAN (classe 0)", uses_gan=True, conditional_label=0),
    StrategySpec("wgan", "WGAN-GP (classe 0)", uses_gan=True, conditional_label=0),
    StrategySpec("smote", "SMOTE (classe 0)"),
    StrategySpec("undersample", "Undersampling da classe 1"),
    StrategySpec("oversample", "Oversampling por repetição (classe 0)"),
]


@dataclass
class ExperimentConfig:
    """Configuração do experimento multietapas."""

    data_flag: str = "breastmnist"
    data_batch_size: int = 128
    latent_dim: int = 100
    gan_epochs: int = 50
    classifier_epochs: int = 5
    num_gan_runs: int = 10
    num_generation_runs: int = 10
    num_classifier_runs: int = 10
    classifier_batch_size: int = 64
    device: Optional[str] = None
    base_seed: int = 2024
    output_dir: Path = Path("balance_class0_runs")


@dataclass
class StageSeeds:
    """Agrupa as sementes de (GAN, geração, classificador) para cada execução."""

    gan_seed: int
    generation_seed: int
    classifier_seed: int


In [5]:
# ---------------------------------------------------------------------------
# Funções utilitárias
# ---------------------------------------------------------------------------

from typing import Optional, Dict, List, Tuple
import os
import random
import numpy as np
import torch
from torch import Tensor
from torch.utils.data import Dataset, TensorDataset


def _resolve_device(device: Optional[str]) -> torch.device:
    """Seleciona automaticamente o dispositivo (CPU ou GPU)."""
    if device is None or device == "auto":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return torch.device(device)


def _seed_everything(seed: int) -> None:
    """Define todas as sementes aleatórias para reprodutibilidade."""
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def _to_scalar(y: Tensor | np.ndarray | int | float) -> int:
    """Converte um tensor, array ou número em escalar inteiro."""
    if isinstance(y, torch.Tensor):
        return int(y.detach().view(-1)[0].item())
    y_np = np.asarray(y)
    return int(y_np.reshape(-1)[0].item())


def count_class_samples(dataset: Dataset) -> Dict[int, int]:
    """Conta o número de amostras por classe em um dataset PyTorch."""
    counts: Dict[int, int] = {}
    for _, label in dataset:
        label_int = _to_scalar(label)
        counts[label_int] = counts.get(label_int, 0) + 1
    return counts


def tensorize_dataset(dataset: Dataset) -> Tuple[Tensor, Tensor]:
    """Converte um ``Dataset`` em tensores (imagens, rótulos)."""
    images: List[Tensor] = []
    labels: List[int] = []

    for img, label in dataset:
        if not isinstance(img, torch.Tensor):
            img = torch.as_tensor(img)
        images.append(img.to(torch.float32))
        labels.append(_to_scalar(label))

    stacked_images = torch.stack(images, dim=0)
    stacked_labels = torch.tensor(labels, dtype=torch.long)
    return stacked_images, stacked_labels


def compute_class_ratio(labels: Tensor) -> float:
    """Retorna a proporção da classe 0 em relação ao total de exemplos."""
    if labels.numel() == 0:
        return float("nan")
    class0 = (labels == 0).sum().item()
    return float(class0 / labels.numel())



In [6]:
# ---------------------------------------------------------------------------
# Manipulação de dados
# ---------------------------------------------------------------------------

from typing import Sequence, Tuple, List, Optional
import time
import torch
from torch import Tensor, nn
from torch.utils.data import TensorDataset


def custom_collate_fn(batch: Sequence[Tuple[Tensor, Tensor]]) -> Tuple[Tensor, Tensor]:
    """
    Função de colagem personalizada para DataLoader.
    Concatena imagens e converte rótulos (tensors ou inteiros) em tensores de inteiros.
    """
    images, labels = zip(*batch)
    xs = torch.stack(images, dim=0)
    ys_list: List[int] = []

    for label in labels:
        if isinstance(label, torch.Tensor):
            if label.numel() == 1:
                ys_list.append(int(label.item()))
            else:
                ys_list.append(int(label.argmax().item()))
        else:
            ys_list.append(int(label))

    ys = torch.tensor(ys_list, dtype=torch.long)
    return xs, ys


def _generate_gan_samples(
    generator: nn.Module,
    *,
    latent_dim: int,
    sample_count: int,
    device: torch.device,
    seed: int,
    conditional_label: Optional[int] = None,
) -> Tensor:
    """Gera amostras sintéticas utilizando um gerador treinado."""
    if sample_count <= 0:
        raise ValueError("sample_count deve ser positivo para geração de amostras")

    _seed_everything(seed)
    generator = generator.to(device).eval()

    noise = torch.randn(sample_count, latent_dim, 1, 1, device=device, dtype=torch.float32)

    with torch.no_grad():
        if conditional_label is not None:
            labels = torch.full((sample_count,), conditional_label, device=device, dtype=torch.long)
            synth_imgs = generator(noise, labels)
        else:
            synth_imgs = generator(noise)

    if synth_imgs.dim() == 3:
        synth_imgs = synth_imgs.unsqueeze(1)

    return synth_imgs.to(torch.float32).cpu()


def build_baseline_dataset(images: Tensor, labels: Tensor) -> Tuple[TensorDataset, int, float, float]:
    """Retorna o dataset original sem modificações."""
    dataset = TensorDataset(images, labels)
    ratio = compute_class_ratio(labels)
    return dataset, 0, ratio, 0.0


def build_gan_balanced_dataset(
    images: Tensor,
    labels: Tensor,
    generator: nn.Module,
    *,
    latent_dim: int,
    synthetic_count: int,
    device: torch.device,
    seed: int,
    conditional_label: Optional[int] = None,
) -> Tuple[TensorDataset, int, float, float]:
    """Balanceia o dataset adicionando amostras sintéticas geradas por GAN."""
    if synthetic_count <= 0:
        return build_baseline_dataset(images, labels)

    start = time.time()
    synth_imgs = _generate_gan_samples(
        generator,
        latent_dim=latent_dim,
        sample_count=synthetic_count,
        device=device,
        seed=seed,
        conditional_label=conditional_label,
    )
    synth_labels = torch.zeros(synthetic_count, dtype=torch.long)

    balanced_images = torch.cat([images, synth_imgs], dim=0)
    balanced_labels = torch.cat([labels, synth_labels], dim=0)
    ratio = compute_class_ratio(balanced_labels)

    dataset = TensorDataset(balanced_images, balanced_labels)
    elapsed = time.time() - start
    return dataset, synthetic_count, ratio, elapsed


def build_smote_dataset(
    images: Tensor,
    labels: Tensor,
    *,
    target_minority: int,
    seed: int,
) -> Tuple[TensorDataset, int, float, float]:
    """Aplica SMOTE na classe 0 até atingir ``target_minority`` exemplos."""
    minority_mask = labels == 0
    minority_images = images[minority_mask]
    minority_count = minority_images.size(0)
    synthetic_count = max(0, target_minority - minority_count)

    if synthetic_count <= 0:
        return build_baseline_dataset(images, labels)

    start = time.time()
    flat_minority = minority_images.view(minority_count, -1).cpu().numpy()

    if minority_count == 1:
        synthetic_flat = np.repeat(flat_minority, synthetic_count, axis=0)
    else:
        from sklearn.neighbors import NearestNeighbors

        k_neighbors = min(5, minority_count)
        nbrs = NearestNeighbors(n_neighbors=k_neighbors)
        nbrs.fit(flat_minority)

        rng = np.random.default_rng(seed)
        base_indices = rng.integers(0, minority_count, size=synthetic_count)
        neighbor_choices = nbrs.kneighbors(flat_minority[base_indices], return_distance=False)
        neighbor_indices = rng.integers(1, neighbor_choices.shape[1], size=synthetic_count)

        diffs = flat_minority[neighbor_choices[np.arange(synthetic_count), neighbor_indices]] - flat_minority[base_indices]
        steps = rng.random((synthetic_count, flat_minority.shape[1]))
        synthetic_flat = flat_minority[base_indices] + steps * diffs

    synthetic_flat = np.clip(synthetic_flat, -1.0, 1.0)
    synth_imgs = torch.from_numpy(synthetic_flat).view(-1, *images.shape[1:]).to(torch.float32)
    synth_labels = torch.zeros(synthetic_count, dtype=torch.long)

    balanced_images = torch.cat([images, synth_imgs], dim=0)
    balanced_labels = torch.cat([labels, synth_labels], dim=0)
    ratio = compute_class_ratio(balanced_labels)
    elapsed = time.time() - start

    dataset = TensorDataset(balanced_images, balanced_labels)
    return dataset, synthetic_count, ratio, elapsed


def build_undersampled_dataset(
    images: Tensor,
    labels: Tensor,
    *,
    minority_count: int,
    seed: int,
) -> Tuple[TensorDataset, int, float, float]:
    """Reduz a classe 1 para a quantidade de exemplos da classe minoritária."""
    start = time.time()
    minority_indices = (labels == 0).nonzero(as_tuple=False).view(-1)
    majority_indices = (labels == 1).nonzero(as_tuple=False).view(-1)

    if majority_indices.numel() <= minority_count:
        return build_baseline_dataset(images, labels)

    g = torch.Generator()
    g.manual_seed(seed)
    perm = torch.randperm(majority_indices.numel(), generator=g)
    selected_majority = majority_indices[perm[:minority_count]]

    selected_indices = torch.cat([minority_indices, selected_majority], dim=0)
    balanced_images = images[selected_indices]
    balanced_labels = labels[selected_indices]
    ratio = compute_class_ratio(balanced_labels)
    elapsed = time.time() - start

    dataset = TensorDataset(balanced_images, balanced_labels)
    return dataset, 0, ratio, elapsed


def build_oversampled_dataset(
    images: Tensor,
    labels: Tensor,
    *,
    target_minority: int,
    seed: int,
) -> Tuple[TensorDataset, int, float, float]:
    """Replica amostras da classe 0 até atingir ``target_minority`` exemplos."""
    minority_indices = (labels == 0).nonzero(as_tuple=False).view(-1)
    minority_count = minority_indices.numel()
    additional = max(0, target_minority - minority_count)

    if additional <= 0 or minority_count == 0:
        return build_baseline_dataset(images, labels)

    start = time.time()
    g = torch.Generator()
    g.manual_seed(seed)
    rand_indices = torch.randint(0, minority_count, (additional,), generator=g)
    sampled_indices = minority_indices[rand_indices]

    synth_images = images[sampled_indices]
    synth_labels = labels[sampled_indices]

    balanced_images = torch.cat([images, synth_images], dim=0)
    balanced_labels = torch.cat([labels, synth_labels], dim=0)
    ratio = compute_class_ratio(balanced_labels)
    elapsed = time.time() - start

    dataset = TensorDataset(balanced_images, balanced_labels)
    return dataset, additional, ratio, elapsed



In [7]:
# ---------------------------------------------------------------------------
# Treinamento
# ---------------------------------------------------------------------------

import time
import numpy as np
from typing import List, Tuple, Optional
import torch
import torch.nn.functional as F
from torch import nn, Tensor
from torch.utils.data import DataLoader


def train_class0_gan(
    train_loader: DataLoader,
    *,
    latent_dim: int,
    gan_epochs: int,
    device: torch.device,
    seed: int,
    img_channels: int,
) -> Tuple[nn.Module, float]:
    """
    Treina um DCGAN para a classe 0 e retorna o gerador treinado e o tempo de execução.

    Args:
        train_loader: DataLoader com amostras reais da classe 0.
        latent_dim: Dimensão do vetor latente de entrada do gerador.
        gan_epochs: Número de épocas de treino do GAN.
        device: Dispositivo (CPU/GPU).
        seed: Semente aleatória para reprodutibilidade.
        img_channels: Número de canais da imagem (ex: 1 para grayscale, 3 para RGB).

    Returns:
        generator: Modelo gerador treinado.
        elapsed: Tempo total de treinamento (em segundos).
    """
    _seed_everything(seed)
    start = time.time()

    generator = DCGenerator(latent_dim=latent_dim, img_channels=img_channels).to(device)
    discriminator = DCDiscriminator(img_channels=img_channels).to(device)

    generator = train_gan_for_class(
        train_loader=train_loader,
        label_target=0,
        G=generator,
        D=discriminator,
        latent_dim=latent_dim,
        num_epochs=gan_epochs,
        device=device,
    ).eval()

    elapsed = time.time() - start
    return generator, elapsed


def train_class0_cgan(
    train_loader: DataLoader,
    *,
    latent_dim: int,
    gan_epochs: int,
    device: torch.device,
    seed: int,
    img_channels: int,
    num_classes: int,
) -> Tuple[nn.Module, float]:
    """Treina um CGAN focado na classe 0."""
    _seed_everything(seed)
    start = time.time()

    generator = CGANGenerator(
        latent_dim=latent_dim,
        num_classes=num_classes,
        img_channels=img_channels,
    ).to(device)
    discriminator = CGANDiscriminator(
        num_classes=num_classes,
        img_channels=img_channels,
    ).to(device)

    generator = train_cgan(
        train_loader=train_loader,
        G=generator,
        D=discriminator,
        latent_dim=latent_dim,
        num_classes=num_classes,
        num_epochs=gan_epochs,
        device=device,
        label_target=0,
    ).eval()

    elapsed = time.time() - start
    return generator, elapsed


def train_class0_wgan(
    train_loader: DataLoader,
    *,
    latent_dim: int,
    gan_epochs: int,
    device: torch.device,
    seed: int,
    img_channels: int,
) -> Tuple[nn.Module, float]:
    """Treina um WGAN-GP restrito à classe 0."""
    _seed_everything(seed)
    start = time.time()

    generator = WGANGPGenerator(latent_dim=latent_dim, img_channels=img_channels).to(device)
    critic = WGANGPCritic(img_channels=img_channels).to(device)

    generator = train_wgangp(
        train_loader=train_loader,
        G=generator,
        D=critic,
        latent_dim=latent_dim,
        num_epochs=gan_epochs,
        device=device,
        label_target=0,
    ).eval()

    elapsed = time.time() - start
    return generator, elapsed


def train_gan_for_strategy(
    strategy: StrategySpec,
    train_loader: DataLoader,
    *,
    latent_dim: int,
    gan_epochs: int,
    device: torch.device,
    seed: int,
    img_channels: int,
    num_classes: int,
) -> Tuple[Optional[nn.Module], float]:
    """Seleciona e treina o gerador adequado para a estratégia informada."""
    if not strategy.uses_gan:
        return None, 0.0

    if strategy.key == "dcgan":
        return train_class0_gan(
            train_loader,
            latent_dim=latent_dim,
            gan_epochs=gan_epochs,
            device=device,
            seed=seed,
            img_channels=img_channels,
        )
    if strategy.key == "cgan":
        return train_class0_cgan(
            train_loader,
            latent_dim=latent_dim,
            gan_epochs=gan_epochs,
            device=device,
            seed=seed,
            img_channels=img_channels,
            num_classes=num_classes,
        )
    if strategy.key == "wgan":
        return train_class0_wgan(
            train_loader,
            latent_dim=latent_dim,
            gan_epochs=gan_epochs,
            device=device,
            seed=seed,
            img_channels=img_channels,
        )

    raise ValueError(f"Estratégia desconhecida para treinamento de GAN: {strategy.key}")


def train_classifier(
    model: nn.Module,
    loader: DataLoader,
    *,
    epochs: int,
    device: torch.device,
) -> None:
    """
    Treina um classificador supervisionado usando Cross-Entropy Loss.

    Args:
        model: Rede neural PyTorch.
        loader: DataLoader com os dados de treino.
        epochs: Número de épocas de treinamento.
        device: Dispositivo (CPU/GPU).
    """
    model.to(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    for _ in range(epochs):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = F.cross_entropy(out, y)
            loss.backward()
            optimizer.step()


def evaluate_classifier(
    model: nn.Module,
    loader: DataLoader,
    *,
    device: torch.device,
) -> Tuple[float, float, float, float, float, int, int, int, int]:
    """
    Avalia o classificador em um conjunto de dados e calcula métricas de desempenho."""
    from sklearn.metrics import (
        accuracy_score,
        confusion_matrix,
        f1_score,
        precision_score,
        recall_score,
        roc_auc_score,
    )

    model.to(device)
    model.eval()

    preds: List[Tensor] = []
    labels: List[Tensor] = []

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = model(x)
            preds.append(out.argmax(dim=1).cpu())
            labels.append(y)

    y_true = torch.cat(labels).numpy()
    y_pred = torch.cat(preds).numpy()

    def _safe_metric(func, default=np.nan):
        try:
            return float(func(y_true, y_pred))
        except Exception:
            return float(default)

    acc = _safe_metric(accuracy_score)
    prec = _safe_metric(lambda yt, yp: precision_score(yt, yp, zero_division=0))
    rec = _safe_metric(lambda yt, yp: recall_score(yt, yp, zero_division=0))
    f1 = _safe_metric(lambda yt, yp: f1_score(yt, yp, zero_division=0))
    auc = _safe_metric(roc_auc_score, default=np.nan)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    return acc, prec, rec, f1, auc, int(tn), int(fp), int(fn), int(tp)



In [8]:
# ---------------------------------------------------------------------------
# Execução do experimento
# ---------------------------------------------------------------------------

import time
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from typing import Dict, List, Tuple

# ---------------------------------------------------------------------------
# Funções auxiliares
# ---------------------------------------------------------------------------


def compute_stage_seeds(
    config: ExperimentConfig,
    strategy_index: int,
    gan_id: int,
    gen_id: int,
    clf_id: int,
) -> StageSeeds:
    """Gera sementes determinísticas para as três fases do experimento."""
    base = config.base_seed + strategy_index * 10_000_000
    gan_seed = base + gan_id
    generation_seed = base + 1_000 * gan_id + gen_id
    classifier_seed = base + 1_000_000 * gan_id + 1_000 * gen_id + clf_id
    return StageSeeds(
        gan_seed=gan_seed,
        generation_seed=generation_seed,
        classifier_seed=classifier_seed,
    )


def prepare_resnet(device: torch.device) -> nn.Module:
    """Cria e retorna uma instância de ResNet18 adaptada para imagens monocanais (1 canal)."""
    model = resnet18(num_classes=2)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    return model.to(device)


# ---------------------------------------------------------------------------
# Pipeline principal de execução
# ---------------------------------------------------------------------------


def run_balance_experiments(config: ExperimentConfig) -> Tuple[pd.DataFrame, Dict[str, pd.DataFrame], Dict[str, object]]:
    """
    Executa o pipeline completo de balanceamento com diferentes estratégias."""
    device = _resolve_device(config.device)

    bundle = load_medmnist_data(
        data_flag=config.data_flag,
        batch_size=config.data_batch_size,
        download=True,
    )

    train_dataset = bundle.train_dataset
    test_dataset = bundle.test_dataset
    train_loader = bundle.train_loader
    eval_loader = DataLoader(
        test_dataset,
        batch_size=config.classifier_batch_size,
        shuffle=False,
        pin_memory=device.type == "cuda",
        drop_last=False,
        collate_fn=custom_collate_fn,
    )

    images, labels = tensorize_dataset(train_dataset)
    base_dataset = TensorDataset(images, labels)

    counts = count_class_samples(base_dataset)
    real_class0 = counts.get(0, 0)
    real_class1 = counts.get(1, 0)
    total_real_samples = len(base_dataset)
    deficit = max(0, real_class1 - real_class0)

    results: List[Dict[str, object]] = []
    img_channels = images.shape[1]
    num_classes = bundle.num_classes if hasattr(bundle, "num_classes") else 2

    strategy_order = {spec.key: idx for idx, spec in enumerate(BALANCING_STRATEGIES, start=1)}
    strategy_names = {spec.key: spec.display_name for spec in BALANCING_STRATEGIES}

    for strategy_idx, strategy in enumerate(BALANCING_STRATEGIES, start=1):
        for gan_run in range(1, config.num_gan_runs + 1):
            seeds_for_gan = compute_stage_seeds(config, strategy_idx, gan_run, 0, 0)
            generator = None
            gan_time = 0.0

            if strategy.uses_gan:
                generator, gan_time = train_gan_for_strategy(
                    strategy,
                    train_loader,
                    latent_dim=config.latent_dim,
                    gan_epochs=config.gan_epochs,
                    device=device,
                    seed=seeds_for_gan.gan_seed,
                    img_channels=img_channels,
                    num_classes=num_classes,
                )

            try:
                for gen_run in range(1, config.num_generation_runs + 1):
                    seeds_for_generation = compute_stage_seeds(
                        config, strategy_idx, gan_run, gen_run, 0
                    )

                    if strategy.key == "baseline":
                        balanced_dataset, synth_count, ratio, generation_time = build_baseline_dataset(
                            images, labels
                        )
                    elif strategy.key in {"dcgan", "cgan", "wgan"}:
                        balanced_dataset, synth_count, ratio, generation_time = build_gan_balanced_dataset(
                            images,
                            labels,
                            generator,
                            latent_dim=config.latent_dim,
                            synthetic_count=deficit,
                            device=device,
                            seed=seeds_for_generation.generation_seed,
                            conditional_label=strategy.conditional_label,
                        )
                    elif strategy.key == "smote":
                        balanced_dataset, synth_count, ratio, generation_time = build_smote_dataset(
                            images,
                            labels,
                            target_minority=real_class1,
                            seed=seeds_for_generation.generation_seed,
                        )
                    elif strategy.key == "undersample":
                        balanced_dataset, synth_count, ratio, generation_time = build_undersampled_dataset(
                            images,
                            labels,
                            minority_count=real_class0,
                            seed=seeds_for_generation.generation_seed,
                        )
                    elif strategy.key == "oversample":
                        balanced_dataset, synth_count, ratio, generation_time = build_oversampled_dataset(
                            images,
                            labels,
                            target_minority=real_class1,
                            seed=seeds_for_generation.generation_seed,
                        )
                    else:
                        raise ValueError(f"Estratégia de balanceamento desconhecida: {strategy.key}")

                    loader = DataLoader(
                        balanced_dataset,
                        batch_size=config.classifier_batch_size,
                        shuffle=True,
                        pin_memory=device.type == "cuda",
                        drop_last=False,
                        collate_fn=custom_collate_fn,
                    )

                    for clf_run in range(1, config.num_classifier_runs + 1):
                        seeds = compute_stage_seeds(
                            config, strategy_idx, gan_run, gen_run, clf_run
                        )
                        _seed_everything(seeds.classifier_seed)

                        model = prepare_resnet(device)

                        train_start = time.time()
                        train_classifier(
                            model,
                            loader,
                            epochs=config.classifier_epochs,
                            device=device,
                        )
                        train_time = time.time() - train_start

                        eval_start = time.time()
                        acc, prec, rec, f1, auc, tn, fp, fn, tp = evaluate_classifier(
                            model,
                            eval_loader,
                            device=device,
                        )
                        eval_time = time.time() - eval_start

                        result = {
                            "strategy": strategy.key,
                            "strategy_display_name": strategy.display_name,
                            "strategy_index": strategy_order[strategy.key],
                            "gan_run_id": gan_run,
                            "generation_run_id": gen_run,
                            "classifier_run_id": clf_run,
                            "ratio": ratio,
                            "acc": acc,
                            "prec": prec,
                            "rec": rec,
                            "f1": f1,
                            "auc": auc,
                            "tn": tn,
                            "fp": fp,
                            "fn": fn,
                            "tp": tp,
                            "synthetic_class0_count": synth_count,
                            "balanced_dataset_size": len(balanced_dataset),
                            "total_real_samples": total_real_samples,
                            "real_class0_count": real_class0,
                            "real_class1_count": real_class1,
                            "gan_training_time_sec": gan_time,
                            "synthetic_generation_time_sec": generation_time,
                            "classifier_training_time_sec": train_time,
                            "classifier_eval_time_sec": eval_time,
                            "gan_seed": seeds_for_gan.gan_seed,
                            "generation_seed": seeds_for_generation.generation_seed,
                            "classifier_seed": seeds.classifier_seed,
                        }

                        results.append(result)
            finally:
                if generator is not None:
                    generator.cpu()
                    del generator
                    if device.type == "cuda":
                        torch.cuda.empty_cache()

    df_results = pd.DataFrame(results)

    summary_tables: Dict[str, pd.DataFrame] = {
        "overall": aggregate_metrics(df_results, []),
        "by_gan": aggregate_metrics(df_results, ["strategy", "gan_run_id"]),
        "by_gan_generation": aggregate_metrics(
            df_results, ["strategy", "gan_run_id", "generation_run_id"]
        ),
        "by_strategy": aggregate_metrics(df_results, ["strategy"]),
    }

    summary = summary_tables["by_strategy"]
    if not summary.empty:
        summary["strategy_display_name"] = summary["strategy"].map(strategy_names)
        summary["strategy_order"] = summary["strategy"].map(strategy_order)
        summary = summary.sort_values("strategy_order").drop(columns=["strategy_order"])
        cols = [
            "strategy_display_name",
            "strategy",
        ] + [c for c in summary.columns if c not in {"strategy_display_name", "strategy"}]
        summary_tables["by_strategy"] = summary[cols]

    metadata: Dict[str, object] = {
        "data_flag": config.data_flag,
        "latent_dim": config.latent_dim,
        "gan_epochs": config.gan_epochs,
        "classifier_epochs": config.classifier_epochs,
        "num_gan_runs": config.num_gan_runs,
        "num_generation_runs": config.num_generation_runs,
        "num_classifier_runs": config.num_classifier_runs,
        "classifier_batch_size": config.classifier_batch_size,
        "data_batch_size": config.data_batch_size,
        "device": str(device),
        "base_seed": config.base_seed,
        "real_class0_count": real_class0,
        "real_class1_count": real_class1,
        "synthetic_needed_for_balance": deficit,
        "total_real_samples": total_real_samples,
        "strategies": [spec.display_name for spec in BALANCING_STRATEGIES],
        "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
    }

    return df_results, summary_tables, metadata


# ---------------------------------------------------------------------------
# Agregação de métricas
# ---------------------------------------------------------------------------


def aggregate_metrics(df: pd.DataFrame, group_cols: List[str]) -> pd.DataFrame:
    """Agrega métricas de classificação, tempo e estatísticas por grupo de execuções."""
    if df.empty:
        return pd.DataFrame(columns=group_cols + [f"{m}_mean" for m in AGGREGATION_FIELDS])

    grouped = df.groupby(group_cols) if group_cols else [((), df)]
    rows: List[Dict[str, object]] = []

    for key, group in grouped:
        if not isinstance(key, tuple):
            key = (key,)

        row: Dict[str, object] = {}
        for idx, col in enumerate(group_cols):
            row[col] = key[idx]

        row["num_rows"] = len(group)

        for metric in AGGREGATION_FIELDS:
            if metric in group:
                row[f"{metric}_mean"] = float(group[metric].mean())
                row[f"{metric}_std"] = float(group[metric].std(ddof=0))

        rows.append(row)

    return pd.DataFrame(rows)



In [9]:
# ---------------------------------------------------------------------------
# Persistência de resultados
# ---------------------------------------------------------------------------

import json
from pathlib import Path
from typing import Dict
import pandas as pd


def save_results(
    df_results: pd.DataFrame,
    summary_tables: Dict[str, pd.DataFrame],
    metadata: Dict[str, object],
    output_dir: Path,
) -> None:
    """
    Salva os resultados do experimento (tabelas e metadados) em arquivos CSV e JSON.

    Args:
        df_results: DataFrame contendo os resultados detalhados de todas as execuções.
        summary_tables: Dicionário com tabelas agregadas de métricas.
        metadata: Dicionário com informações da configuração e execução do experimento.
        output_dir: Diretório de saída onde os arquivos serão salvos.

    Saídas:
        - balance_class0_results.csv: Resultados completos.
        - summary_*.csv: Tabelas de resumo (overall, por GAN, etc.).
        - metadata.json: Informações descritivas e parâmetros da execução.
    """
    # Cria o diretório de saída, se necessário
    output_dir.mkdir(parents=True, exist_ok=True)

    # Salva resultados completos
    results_path = output_dir / "balance_class0_results.csv"
    df_results.to_csv(results_path, index=False)

    # Salva tabelas agregadas
    for name, df in summary_tables.items():
        df.to_csv(output_dir / f"summary_{name}.csv", index=False)

    # Salva metadados como JSON
    with open(output_dir / "metadata.json", "w", encoding="utf-8") as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)


## Execução do experimento. Ajuste os hiperparâmetros abaixo conforme necessário. Dependendo dos valores escolhidos(as execuções padrão são 10×10×10), o processo pode ser bastante demorado. Para testes rápidos,reduza os contadores de repetições e épocas.

In [10]:
# ---------------------------------------------------------------------------
# Exemplo de configuração enxuta para testes rápidos
# ---------------------------------------------------------------------------

config = ExperimentConfig(
    data_flag="breastmnist",
    data_batch_size=128,
    latent_dim=100,
    gan_epochs=50,
    classifier_epochs=1,
    num_gan_runs=1,
    num_generation_runs=1,
    num_classifier_runs=1,
    classifier_batch_size=64,
    device="auto",
    base_seed=2024,
    output_dir=Path("balance_class0_runs_demo"),
)

# ---------------------------------------------------------------------------
# Execução opcional do pipeline completo
# ---------------------------------------------------------------------------

# Descomente as linhas abaixo para executar todo o experimento.
# Isso irá:
# 1. Treinar os geradores necessários para cada estratégia (quando aplicável).
# 2. Gerar datasets balanceados para cada abordagem.
# 3. Treinar e avaliar diversas vezes o classificador ResNet-18.
# 4. Salvar os resultados e metadados no diretório especificado.

df_results, summary_tables, metadata = run_balance_experiments(config)
save_results(df_results, summary_tables, metadata, config.output_dir)



/home/mahlow/anaconda3/envs/my_env/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Using downloaded and verified file: /home/mahlow/.medmnist/breastmnist.npz
Using downloaded and verified file: /home/mahlow/.medmnist/breastmnist.npz


/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))


Epoch 1/50 - Loss D: 1.296 | Loss G: 0.868
Epoch 2/50 - Loss D: 0.813 | Loss G: 1.175
Epoch 3/50 - Loss D: 0.567 | Loss G: 1.496
Epoch 4/50 - Loss D: 0.446 | Loss G: 1.780
Epoch 5/50 - Loss D: 0.351 | Loss G: 2.056
Epoch 6/50 - Loss D: 0.261 | Loss G: 2.259
Epoch 7/50 - Loss D: 0.260 | Loss G: 2.368
Epoch 8/50 - Loss D: 0.194 | Loss G: 2.464
Epoch 9/50 - Loss D: 0.203 | Loss G: 2.556
Epoch 10/50 - Loss D: 0.139 | Loss G: 2.787
Epoch 11/50 - Loss D: 0.130 | Loss G: 3.005
Epoch 12/50 - Loss D: 0.105 | Loss G: 3.075
Epoch 13/50 - Loss D: 0.101 | Loss G: 3.220
Epoch 14/50 - Loss D: 0.084 | Loss G: 3.326
Epoch 15/50 - Loss D: 0.080 | Loss G: 3.334
Epoch 16/50 - Loss D: 0.076 | Loss G: 3.418
Epoch 17/50 - Loss D: 0.070 | Loss G: 3.472
Epoch 18/50 - Loss D: 0.062 | Loss G: 3.549
Epoch 19/50 - Loss D: 0.054 | Loss G: 3.752
Epoch 20/50 - Loss D: 0.053 | Loss G: 3.678
Epoch 21/50 - Loss D: 0.204 | Loss G: 3.052
Epoch 22/50 - Loss D: 0.122 | Loss G: 3.750
Epoch 23/50 - Loss D: 0.111 | Loss G: 3.7

/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))
/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))


Epoch 1/50 | Loss D: -1.2006 | Loss G: 0.4956
Epoch 2/50 | Loss D: -4.5872 | Loss G: 1.0913
Epoch 3/50 | Loss D: -6.0631 | Loss G: 1.5362
Epoch 4/50 | Loss D: -7.9754 | Loss G: 2.1832
Epoch 5/50 | Loss D: -9.2319 | Loss G: 2.6307
Epoch 6/50 | Loss D: -10.1630 | Loss G: 3.4293
Epoch 7/50 | Loss D: -12.2373 | Loss G: 3.9435
Epoch 8/50 | Loss D: -13.9389 | Loss G: 4.5702
Epoch 9/50 | Loss D: -13.0932 | Loss G: 4.9185
Epoch 10/50 | Loss D: -15.3010 | Loss G: 5.0502
Epoch 11/50 | Loss D: -16.1803 | Loss G: 5.8140
Epoch 12/50 | Loss D: -17.5843 | Loss G: 6.2687
Epoch 13/50 | Loss D: -17.9028 | Loss G: 6.2935
Epoch 14/50 | Loss D: -19.6762 | Loss G: 6.8740
Epoch 15/50 | Loss D: -19.0058 | Loss G: 7.2204
Epoch 16/50 | Loss D: -19.8783 | Loss G: 7.1760
Epoch 17/50 | Loss D: -21.1980 | Loss G: 7.6468
Epoch 18/50 | Loss D: -22.4456 | Loss G: 7.3861
Epoch 19/50 | Loss D: -23.1469 | Loss G: 8.1046
Epoch 20/50 | Loss D: -23.8352 | Loss G: 8.5580
Epoch 21/50 | Loss D: -22.0286 | Loss G: 8.4055
Epoch 

/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))
/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))
/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ys_list.append(int(label))
/tmp/ipykernel_595721/3240726967.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and wi

In [11]:
# Exiba os resultados agregados por estratégia após executar o pipeline
summary_tables["by_strategy"]



,strategy_display_name,strategy,num_rows,acc_mean,acc_std,prec_mean,prec_std,rec_mean,rec_std,f1_mean,...,synthetic_class0_count_mean,synthetic_class0_count_std,balanced_dataset_size_mean,balanced_dataset_size_std,total_real_samples_mean,total_real_samples_std,real_class0_count_mean,real_class0_count_std,real_class1_count_mean,real_class1_count_std
0,ResNet sem balanceamento,baseline,1,0.717949,0.0,0.769231,0.0,0.877193,0.0,0.819672,...,0.0,0.0,546.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0
2,DCGAN (classe 0),dcgan,1,0.730769,0.0,0.730769,0.0,1.000000,0.0,0.844444,...,252.0,0.0,798.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0
1,CGAN (classe 0),cgan,1,0.634615,0.0,0.901408,0.0,0.561404,0.0,0.691892,...,252.0,0.0,798.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0
6,WGAN-GP (classe 0),wgan,1,0.730769,0.0,0.740000,0.0,0.973684,0.0,0.840909,...,252.0,0.0,798.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0
4,SMOTE (classe 0),smote,1,0.487179,0.0,0.904762,0.0,0.333333,0.0,0.487179,...,252.0,0.0,798.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0
5,Undersampling da classe 1,undersample,1,0.371795,0.0,0.900000,0.0,0.157895,0.0,0.268657,...,0.0,0.0,294.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0
3,Oversampling por repetição (classe 0),oversample,1,0.384615,0.0,0.875000,0.0,0.184211,0.0,0.304348,...,252.0,0.0,798.0,0.0,546.0,0.0,147.0,0.0,399.0,0.0


## Utilitário de linha de comando (opcional)As funções abaixo permitem reutilizar o parser de argumentos do script original, casoqueira invocar o notebook via `papermill` ou semelhante.

In [ ]:
# ---------------------------------------------------------------------------
# Parser de argumentos para execução programática
# ---------------------------------------------------------------------------

import argparse
from pathlib import Path


def parse_args() -> ExperimentConfig:
    """
    Cria e interpreta argumentos de linha de comando para configurar o experimento.

    Em um notebook, o parse_args([]) evita erros por ausência de argumentos
    e retorna um objeto ExperimentConfig com valores padrão ou definidos manualmente.

    Returns:
        ExperimentConfig: Objeto de configuração preenchido com os parâmetros fornecidos.
    """
    parser = argparse.ArgumentParser(
        description="Run repeated class-0 balancing experiments with DCGAN and ResNet-18",
    )

    # Argumentos principais
    parser.add_argument("--data-flag", default="breastmnist", help="MedMNIST dataset flag")
    parser.add_argument("--data-batch-size", type=int, default=128, help="Batch size for the GAN data loader")
    parser.add_argument("--latent-dim", type=int, default=100, help="Latent dimension for the GAN")
    parser.add_argument("--gan-epochs", type=int, default=50, help="Number of epochs for GAN training")
    parser.add_argument(
        "--classifier-epochs",
        type=int,
        default=5,
        help="Number of epochs for each ResNet-18 training run",
    )
    parser.add_argument("--num-gan-runs", type=int, default=10, help="How many times to retrain the GAN")
    parser.add_argument(
        "--num-generation-runs",
        type=int,
        default=10,
        help="How many synthetic datasets to generate per GAN training",
    )
    parser.add_argument(
        "--num-classifier-runs",
        type=int,
        default=10,
        help="How many classifier trainings per synthetic dataset",
    )
    parser.add_argument(
        "--classifier-batch-size",
        type=int,
        default=64,
        help="Batch size for the ResNet-18 training and evaluation",
    )
    parser.add_argument(
        "--device",
        default="auto",
        help="Device to use (cuda, cpu or auto)",
    )
    parser.add_argument("--base-seed", type=int, default=2024, help="Base seed for reproducibility")
    parser.add_argument(
        "--output-dir",
        type=Path,
        default=Path("balance_class0_runs"),
        help="Directory where CSV and metadata files will be stored",
    )

    # parse_args([]) evita conflitos de CLI em ambientes interativos
    args = parser.parse_args([])

    return ExperimentConfig(
        data_flag=args.data_flag,
        data_batch_size=args.data_batch_size,
        latent_dim=args.latent_dim,
        gan_epochs=args.gan_epochs,
        classifier_epochs=args.classifier_epochs,
        num_gan_runs=args.num_gan_runs,
        num_generation_runs=args.num_generation_runs,
        num_classifier_runs=args.num_classifier_runs,
        classifier_batch_size=args.classifier_batch_size,
        device=args.device,
        base_seed=args.base_seed,
        output_dir=args.output_dir,
    )
